In [ ]:
# You need to do some weird setup stuff:
# 

In [74]:
!ls -latr waifu-diffusion/logs/

total 4
drwxr-xr-x 16 root root 4096 Sep 18 16:27 ..
drwxr-xr-x  2 root root    6 Sep 18 16:38 .ipynb_checkpoints
drwxr-xr-x  6 root root   70 Sep 18 17:45 2022-09-18T17-44-13_aesthetic
drwxr-xr-x  4 root root   69 Sep 18 17:48 .


In [75]:
!ls -latr waifu-diffusion/logs/2022-09-18T17-44-13_aesthetic/checkpoints

total 0
drwxr-xr-x 2 root root  6 Sep 18 17:44 .
drwxr-xr-x 6 root root 70 Sep 18 17:45 ..


In [68]:
!rm waifu-diffusion/logs/2022-09-18T17-07-00_aesthetic/checkpoints/epoch=000008-v1.ckpt

In [70]:
!cp waifu-diffusion/logs/2022-09-18T17-07-00_aesthetic/checkpoints/last.ckpt pokediffusion_epoch_3.ckpt

In [31]:
import os
import torch

def prune_it(p, keep_only_ema=False):
    print(f"prunin' in path: {p}")
    size_initial = os.path.getsize(p)
    nsd = dict()
    sd = torch.load(p, map_location="cpu")
    print(sd.keys())
    for k in sd.keys():
        if k != "optimizer_states":
            nsd[k] = sd[k]
    else:
        print(f"removing optimizer states for path {p}")
    if "global_step" in sd:
        print(f"This is global step {sd['global_step']}.")
    if keep_only_ema:
        sd = nsd["state_dict"].copy()
        # infer ema keys
        ema_keys = {k: "model_ema." + k[6:].replace(".", ".") for k in sd.keys() if k.startswith("model.")}
        new_sd = dict()

        for k in sd:
            if k in ema_keys:
                new_sd[k] = sd[ema_keys[k]].half()
            elif not k.startswith("model_ema.") or k in ["model_ema.num_updates", "model_ema.decay"]:
                new_sd[k] = sd[k].half()

        assert len(new_sd) == len(sd) - len(ema_keys)
        nsd["state_dict"] = new_sd
    else:
        sd = nsd['state_dict'].copy()
        new_sd = dict()
        for k in sd:
            new_sd[k] = sd[k].half()
        nsd['state_dict'] = new_sd

    fn = f"{os.path.splitext(p)[0]}_pruned.ckpt" if not keep_only_ema else f"{os.path.splitext(p)[0]}-ema-pruned.ckpt"
    print(f"saving pruned checkpoint at: {fn}")
    torch.save(nsd, fn)
    newsize = os.path.getsize(fn)
    MSG = f"New ckpt size: {newsize*1e-9:.2f} GB. " + \
          f"Saved {(size_initial - newsize)*1e-9:.2f} GB by removing optimizer states"
    if keep_only_ema:
        MSG += " and non-EMA weights"
    print(MSG)

In [71]:
prune_it('pokediffusion_epoch_3.ckpt')

prunin' in path: pokediffusion_epoch_3.ckpt
dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])
removing optimizer states for path pokediffusion_epoch_3.ckpt
This is global step 473500.
saving pruned checkpoint at: pokediffusion_epoch_3-pruned.ckpt
New ckpt size: 3.85 GB. Saved 10.73 GB by removing optimizer states


In [35]:
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!sudo apt install unzip -y
!unzip awscliv2.zip
!sudo ./aws/install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.8M  100 44.8M    0     0  26.7M      0  0:00:01 --:--:--  0:00:01 26.6M  0  36.6M      0  0:00:01  0:00:01 --:--:-- 36.6M
/bin/bash: unzip: command not found
sudo: ./aws/install: command not found


In [40]:
!mkdir ~/.aws
# Upload your ~/.aws/credentials now

!aws configure set default.s3.max_concurrent_requests 3
!aws configure set default.s3.multipart_chunksize 128MB


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: the following arguments are required: command



In [72]:
!aws configure set default.s3.max_concurrent_requests 3
!aws configure set default.s3.multipart_chunksize 128MB

In [73]:
!aws s3 cp --endpoint-url https://e8c8a19b99810a33b983068eabf3df2f.r2.cloudflarestorage.com/sd-finetune pokediffusion_epoch_3-pruned.ckpt s3://pokediffusion_ckpts

upload: ./pokediffusion_epoch_3-pruned.ckpt to s3://pokediffusion_ckpts/pokediffusion_epoch_3-pruned.ckpt


In [ ]:
!rm 